In [16]:
import numpy as np
from docplex.mp.model import Model

# from docplex.mp.callbacks.cb_mixin import ConstraintCallbackMixin
# from docplex.mp.callbacks.cb_mixin import LazyConstraintCallback
import matplotlib.pyplot as plt

In [1]:
import cplex
from cplex.callbacks import UserCutCallback, LazyConstraintCallback
import multiprocessing
import time
import sys
import os
import itertools
import cplex
import numpy as np

In [2]:
# ユークリッド距離の計算
import numpy as np

# 修正後のユークリッド距離の計算関数
def compute_distances(demand_points, candidate_sites):
    D, J = len(demand_points), len(candidate_sites)  # ここで D, J を定義
    distances = np.zeros((D, J))
    for d in range(D):
        for j in range(J):
            distances[d, j] = np.sqrt(
                (demand_points[d][0] - candidate_sites[j][0]) ** 2
                + (demand_points[d][1] - candidate_sites[j][1]) ** 2
            )
    return distances


In [3]:
def compute_Ui_L(wij_matrix, J_L):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    return wij_matrix[:, list(J_L)].sum(axis=1)


def compute_Ui_F(wij_matrix, J_F):
    """
    リーダーの既存施設による U_i^L を計算する関数

    Parameters:
        wij_matrix (np.array): D × J の w_ij の重み行列
        J_L (set): リーダーが既に持っている施設のインデックス集合

    Returns:
        np.array: 各需要点 i に対する U_i^L のベクトル
    """
    return wij_matrix[:, list(J_F)].sum(axis=1)


In [4]:
def compute_L(h_i, Ui_L, Ui_F, wij, x, y):
    """
    関数 L(x, y) を計算する

    Parameters:
        h (np.array): 需要点ごとの人口密度ベクトル (D,)
        Ui_L (np.array): 各需要点におけるリーダーの影響度 (D,)
        Ui_F (np.array): 各需要点におけるフォロワーの影響度 (D,)
        wij (np.array): 需要点と施設候補の重み行列 (D, J)
        x (np.array): リーダーが選択した施設配置 (J,)
        y (np.array): フォロワーが選択した施設配置 (J,)

    Returns:
        float: L(x, y) の計算結果
    """
    numerator = Ui_L + (wij @ x)  # 分子: リーダーの影響度 + 選択した施設の影響
    denominator = Ui_L + Ui_F + (wij @ np.maximum(x, y))  # 分母: 総合影響度

    return np.sum(h_i * (numerator / denominator))


### 大枠から作っていきますか

30秒たっても解けない場合は強制ストップ

In [5]:
def run_solve_s_cflp():
    """solve_s_cflp() を実行し、出力を 'output.log' に保存"""
    with open("output.log", "w") as f:
        sys.stdout = f  # `print()` の出力をファイルにリダイレクト
        sys.stderr = f  # エラーメッセージも保存
        try:
            solve_s_cflp()  # solve_s_cflp を実行
        except Exception as e:
            print(f"⚠️ solve_s_cflp() failed with error: {e}")
        finally:
            sys.stdout = sys.__stdout__  # 元の標準出力に戻す

def solve_with_timeout(timeout=30):
    """solve_s_cflp() を指定時間内に実行し、超過したら強制終了"""
    # **出力ファイルを事前に作成**
    with open("output.log", "w") as f:
        f.write("=== Starting solve_s_cflp() ===\n")

    process = multiprocessing.Process(target=run_solve_s_cflp)
    process.start()
    process.join(timeout)  # 指定時間（秒）待機

    if process.is_alive():
        print("⚠️ solve_s_cflp() was forcibly stopped due to timeout (30s).")
        process.terminate()  # プロセスを強制終了
        process.join()  # プロセスの後処理
        with open("output.log", "a") as f:
            f.write("\n⚠️ solve_s_cflp() was forcibly stopped due to timeout (30s).\n")


In [6]:
def solve_s_cflp():
    """S-CFLPのBranch-and-Cutフレームワークを実装"""
    global F, BestSol, theta_LB

    # **初期化**
    initialize_problem()

    # **Branch-and-Cut のループ**
    while F:
        # F から最新の Cplex 問題オブジェクトを取得
        problem = F.pop()
        # **連続緩和問題を解く**
        incumbent_solution = solve_relaxation(problem)

        # **分離問題を解く**
        y_star = separation_problem(incumbent_solution)

        # **カットを追加するかチェック**
        if should_add_cut(incumbent_solution, y_star):
            new_problem = add_cuts(incumbent_solution, y_star, problem)
            
            print(len(F))
            F.append(new_problem)
        else:
            # **最適整数解なら終了**
            if is_integer_solution(incumbent_solution, theta_LB):
                update_best_solution(incumbent_solution)
                break
            else:
                # **分枝処理を行う**
                problem_0, problem_1 = branch_and_bound(incumbent_solution)
                F.append(problem_0)
                F.append(problem_1)

In [7]:
def initialize_problem():
    """Branch-and-Cutフレームワークの初期化"""
    global F, BestSol, theta_LB  # グローバル変数として明示

    problem = cplex.Cplex()

    # **最大化問題**
    problem.objective.set_sense(problem.objective.sense.maximize)

    # **変数 x_j の追加**
    x_names = [f"x{j}" for j in range(J)]
    problem.variables.add(names=x_names, types=["C"] * J, lb=[0] * J, ub=[1] * J, obj=[0] * J)

    # **目的変数 θ の追加**
    problem.variables.add(names=["theta"], types=["C"], lb=[0], ub=[1], obj=[1])

    # **施設数制約**
    problem.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=x_names, val=[1.0] * J)],
        senses=["E"], rhs=[p], names=["facility_limit"]
    )

    # **F の初期化**
    F = [problem]  # 問題をリストに追加
    BestSol = None  # 最良解リセット
    theta_LB = 0  # 下限値リセット

In [8]:
def solve_relaxation(problem):
    """
    連続緩和問題を解き、インカンベント解 (x_hat, theta_hat) を取得する。

    Parameters:
        F (list): 定式化のセット（Cplex 問題オブジェクトのリスト）

    Returns:
        dict: インカンベント解 {'x': np.array, 'theta': float}
    """

    # 問題を解く
    problem.solve()

    # CPLEX の解の取得（変数名のアンダースコアを削除）
    x_values = np.array(
        problem.solution.get_values([f"x{j}" for j in range(J)])
    )  # "x_0" → "x0"
    theta_value = problem.solution.get_values("theta")

    # インカンベント解を辞書形式で返す
    return {"x": x_values, "theta": theta_value}

In [9]:
def separation_problem(incumbent_solution):
    """
    分離問題を解く。Proposition 5 に基づき、最適なフォロワーの施設選択 ŷ を決定する。

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}

    Returns:
        np.array: 最適なフォロワーの施設配置 y_hat (J,)
    """
    # インカンベント解 (x_hat)
    x_hat = incumbent_solution["x"]

    # 定数 a_i(x_hat) の計算
    a_i = Ui_L + (wij_matrix @ x_hat)

    # 定数 w_i^L(x_hat) および w_i^U(x_hat) の計算
    w_i_L = np.min(Ui_F[:, np.newaxis] + wij_matrix * (1 - x_hat), axis=1)
    w_i_U = np.max(Ui_F[:, np.newaxis] + wij_matrix * (1 - x_hat), axis=1)

    # β(x_hat) の計算
    beta_hat = np.sum(
        h_i[:, np.newaxis]
        * (
            (a_i[:, np.newaxis] * wij_matrix * (1 - x_hat))
            / (
                (a_i[:, np.newaxis] + w_i_U[:, np.newaxis])
                * (a_i[:, np.newaxis] + w_i_L[:, np.newaxis])
            )
        ),
        axis=0,
    )

    # β(x_hat) の降順ソート
    sorted_indices = np.argsort(-beta_hat)  # 降順ソート

    # フォロワーの最適な選択 y_hat
    y_hat = np.zeros(J)
    y_hat[sorted_indices[:r]] = 1  # 上位 r 個を選択

    return y_hat

In [10]:
def should_add_cut(incumbent_solution, y_star):
    """
    カットを追加すべきか判定

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
        y_star (np.array): フォロワーの最適な施設配置 y_hat (J,)

    Returns:
        bool: カットを追加すべきなら True, そうでなければ False
    """
    # インカンベント解 (x_hat, theta_hat)
    x_hat = incumbent_solution["x"]
    theta_hat = incumbent_solution["theta"]

    # L(x_hat, y_star) を計算
    L_value = compute_L(h_i, Ui_L, Ui_F, wij_matrix, x_hat, y_star)

    # カット追加判定
    return theta_hat > L_value

In [11]:
def generate_subsets(J):
    """
    J のすべての部分集合 S ⊆ J を生成する（全探索）

    Parameters:
        J (int): 施設候補の数（candidate_sites の長さ）

    Returns:
        list: すべての部分集合 S のリスト（各要素は set 型）
    """
    candidate_indices = set(range(J))  # 修正: `J` を `int` として受け取る
    subsets = []

    # J のすべての部分集合を生成（空集合から J まで）
    for rr in range(len(candidate_indices) + 1):
        for subset in itertools.combinations(candidate_indices, rr):
            subsets.append(set(subset))  # `set` 型で格納

    return subsets


def compute_L_Y(S, Y):
    """
    L_Y(S) を計算する

    Parameters:
        S (set): リーダーが開設する施設の部分集合
        Y (set): フォロワーが開設する施設の集合

    Returns:
        float: L_Y(S)
    """
    # この関数はあってそう
    # print("=== compute_L_Y ===")
    # print(f"S: {S}")
    # print(f"Y: {Y}")
    # print(f"wij_matrix[:, list(S)] : {wij_matrix[:, list(S)]}")
    # print(f"wij_matrix : {wij_matrix}")
    # print("=====================")

    # wij_matrix[:, list(S)]は D × |S| の行列を返却

    f_i_Y = (Ui_L[:, np.newaxis] + wij_matrix[:, list(S)].sum(axis=1)) / (
        Ui_L[:, np.newaxis]
        + Ui_F[:, np.newaxis]
        + wij_matrix[:, list(S.union(Y))].sum(axis=1)
    )
    return np.sum(h_i * f_i_Y)


def compute_rho_Y(S, k, Y):
    """
    rho_Y(S, k) を計算する

    Parameters:
        S (set): リーダーが開設する施設の部分集合
        k (int): 追加する施設のインデックス
        Y (set): フォロワーの施設配置

    Returns:
        float: rho_Y(S, k)
    """
    return compute_L_Y(S.union({k}), Y) - compute_L_Y(S, Y)

In [ ]:
def add_cuts(incumbent_solution, y_star, problem):
    """
    カット（制約）を追加する。Algorithm 1 に基づき、サブモジュラー不等式 (8) を適用し、
    強化した定式化を F に戻す。

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
        y_star (np.array): フォロワーの最適な施設配置 y_hat (J,)
    """
    # インカンベント解 (x_hat, theta_hat)
    x_hat = incumbent_solution["x"]
    theta_hat = incumbent_solution["theta"]

    # フォロワーが選んだ施設の集合 Y
    Y = {j for j in range(J) if y_star[j] == 1}
    print(f"\n🔍 [DEBUG] in add_cuts | Y (フォロワーの施設集合): {Y}")

    # 候補施設の集合
    J_set = set(range(J))
    print(f"📌 [DEBUG] J_set (リーダーの候補施設集合): {J_set}")

    # 最も違反する S を探索
    min_cut_value = float("inf")
    best_S = None

    for S in generate_subsets(J):
        S_set = set(S)

        # 定数項・補正値の計算
        rho_sum_1 = sum(compute_rho_Y(J_set - {k}, k, Y) for k in S_set)
        rho_sum_2 = sum(compute_rho_Y(J_set - {k}, k, Y) * x_hat[k] for k in S_set)
        rho_sum_3 = sum(compute_rho_Y(S_set, k, Y) * x_hat[k] for k in (J_set - S_set))

        cut_value = compute_L_Y(S_set, Y) - rho_sum_1 + rho_sum_2 + rho_sum_3

        # デバッグ用出力 (S_set の各 k について明示)
        print(f"\n🔄 [DEBUG] Checking subset S: {S_set}")
        print(f"   L_Y(S_set, Y): {compute_L_Y(S_set, Y):.4f}")
        
        print(f"   Σ rho_Y(J\\k, k, Y) (定数項補正):")
        for k in S_set:
            print(f"      k={k}, rho_Y(J\\{k}, k, Y): {compute_rho_Y(J_set - {k}, k, Y):.4f}")

        print(f"   Σ rho_Y(J\\k, k, Y) * x_hat[k]:")
        for k in S_set:
            print(f"      k={k}, rho_Y(J\\{k}, k, Y) * x_hat[k]: {compute_rho_Y(J_set - {k}, k, Y) * x_hat[k]:.4f}")

        print(f"   Σ rho_Y(S, k, Y) * x_hat[k]:")
        for k in (J_set - S_set):
            print(f"      k={k}, rho_Y(S, k, Y) * x_hat[k]: {compute_rho_Y(S_set, k, Y) * x_hat[k]:.4f}")

        print(f"   ➡️ Computed cut_value: {cut_value:.4f}")

        # **cut_value の最小値を更新するか判定**
        if cut_value <= min_cut_value:
            print(f"   ✅ [UPDATE] New Best S Found! (cut_value={cut_value:.4f} <= min_cut_value={min_cut_value:.4f})")
            min_cut_value = cut_value
            best_S = S_set
        else:
            print(f"   ❌ [NO UPDATE] cut_value={cut_value:.4f} > min_cut_value={min_cut_value:.4f}")

    # `best_S` の出力を追加
    print("\n=== ✅ Best S Found ===")
    if best_S:
        print(f"   S* = {sorted(best_S)} (最も違反する部分集合)")
    else:
        print("   S* = None (カットを追加する必要なし)")
    print("====================\n")

    # CPLEX 問題の取得
    # 引数で渡す
    # problem = F[-1]

    # サブモジュラー不等式 (8) に基づくカットを追加
    if best_S is not None:
        x_coeffs_s = {k: compute_rho_Y(J_set - {k}, k, Y) for k in best_S}  # S 内の x の係数
        x_coeffs_js = {k: compute_rho_Y(best_S, k, Y) for k in (J_set - best_S)}  # J\S の x の係数

        # 定数項の計算
        constant_term = compute_L_Y(best_S, Y) - sum(
            compute_rho_Y(J_set - {k}, k, Y) for k in best_S
        )

        # `rhs_value` は符号を反転
        rhs_value = constant_term

        # CPLEX に渡す SparsePair の作成
        submodular_cut_expr = cplex.SparsePair(
            ind=["theta"] + [f"x{k}" for k in J_set],  # θ も変数リストに追加
            val=[-1.0] + list(x_coeffs_s.values()) + list(x_coeffs_js.values()),  # -θ の係数も追加
        )

        # 制約の数式を作成
        constraint_str = f"-θ"
        for k, coeff in x_coeffs_s.items():
            constraint_str += f" + ({coeff:.4f})x_{k}"
        for k, coeff in x_coeffs_js.items():
            constraint_str += f" + ({coeff:.4f})x_{k}"
        constraint_str += f" ≥ {-rhs_value:.4f}"

        print("\n=== ➕ Added Submodular Cut ===")
        print(f"   {constraint_str}")
        print("==============================\n")

        # CPLEX に制約を追加
        problem.linear_constraints.add(
            lin_expr=[submodular_cut_expr],  # 制約の左辺
            senses=["G"],  # "G" は ≥ を意味する (左辺 >= 右辺)
            rhs=[rhs_value],  # 右辺の値 (定数項)
            names=[f"cut_submodular_{len(problem.linear_constraints.get_names())}"],
        )


    # 強化した定式化を F に戻す
    return problem


# ↑要検討。間違ってる可能性の方が高い

とりあえず次に進む

In [13]:
def is_integer_solution(incumbent_solution, theta_LB, tol=1e-5):
    """
    インカンベント解のリーダー施設配置 x が整数 (0 or 1) であり、
    かつ目的関数値 theta が θ_LB を超えているか判定。

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
        theta_LB (float): 現在の目的関数の下界
        tol (float): 数値誤差の許容範囲（デフォルト 1e-5）

    Returns:
        bool: すべての x_j が整数であり、かつ θ > θ_LB なら True, そうでなければ False
    """
    # 条件1: 目的関数値 θ が θ_LB を超えているか
    if incumbent_solution["theta"] <= theta_LB:
        return False

    # 条件2: x のすべての要素が 0 or 1 の整数か
    x_values = incumbent_solution["x"]
    if np.all(np.abs(x_values - np.round(x_values)) < tol):  # すべて整数に近い
        return True
    else:
        return False

In [14]:
def update_best_solution(incumbent_solution):
    """
    インカンベント解 (x, theta) を用いて最良解 BestSol を更新し、
    目的関数の下界 θ_LB を更新する。

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
    """
    global BestSol, θ_LB  # グローバル変数を更新

    # BestSol の更新（x のコピーを保存）
    BestSol = incumbent_solution["x"].copy()

    # θ_LB の更新
    θ_LB = incumbent_solution["theta"]

    print("\n=== Best Solution Updated ===")
    print(f"BestSol: {BestSol}")
    print(f"θ_LB: {θ_LB:.4f}")
    print("==============================\n")

In [15]:
def branch_and_bound(incumbent_solution):
    """
    分枝限定法を適用し、最も整数から遠い x_j を固定して
    2つの新しい定式化を作成し、リスト F に追加する。

    Parameters:
        incumbent_solution (dict): インカンベント解 {'x': np.array, 'theta': float}
    """
    global F  # 定式化のリスト F を更新

    x_hat = incumbent_solution["x"]

    # **最も整数から遠い x_j を選択**
    fractional_indices = np.where((x_hat > 1e-5) & (x_hat < 1 - 1e-5))[0]
    if len(fractional_indices) == 0:
        print("⚠️ No fractional x found. Branching is not needed.")
        return

    # **最大誤差のある x_j を選択**
    j_star = fractional_indices[np.argmax(np.abs(x_hat[fractional_indices] - 0.5))]

    print("\n=== 🔀 Branching on x_{} ===".format(j_star))
    print("   Fractional value: x_{} = {:.4f}".format(j_star, x_hat[j_star]))
    print("==============================\n")

    # **現在の定式化をコピー**
    problem = F[-1]  # 現在の問題
    problem_0 = problem.copy()  # x_j = 0 の場合
    problem_1 = problem.copy()  # x_j = 1 の場合

    # **🔹 制約 x_j = 0 を追加**
    problem_0.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=[f"x{j_star}"], val=[1.0])],
        senses=["E"],  # "=" 制約
        rhs=[0.0],
        names=[f"branch_x_{j_star}_0"],
    )

    # **🔹 制約 x_j = 1 を追加**
    problem_1.linear_constraints.add(
        lin_expr=[cplex.SparsePair(ind=[f"x{j_star}"], val=[1.0])],
        senses=["E"],  # "=" 制約
        rhs=[1.0],
        names=[f"branch_x_{j_star}_1"],
    )

    # **新しい定式化をリスト F に追加**
    return problem_0, problem_1


